In [14]:
class Node:
    def __init__(self,position,father):
        self.position = position 
        self.father = father 
        
    def __repr__(self):
        return str(self.position[0])+" "+str(self.position[1])
    
class NodeStar:
    def __init__(self,position,father,g,h,f=None):
        self.position = position
        self.father = father
        self.h = h
        self.g = g
        self.f = g+h
        
    def __repr__(self):
        return str(self.position[0])+" "+str(self.position[1])
    
    def __lt__(self,other):
        return self.f < other.f
    
    def hn(self,goal):#Metodo que retorna el valor heuristico con la distancia Manhattan
        return abs(self.position[0] - goal[0]) + abs(self.position[1] - goal[1])

#Function to create a list of nodes
def Vertices(L): 
    Vertices = []
    start = []
    goal = []
    for i in range(len(L)):
        for j in range(len(L[i])):
            Ver = []
            Ver.append(i)
            Ver.append(j)
            if L[i][j] != '%': #Wall
                Vertices.append(Ver)
                if L[i][j] == 'P': #Pacman Position
                    start = Ver[:]
                elif L[i][j] == '.': #Fruit
                    goal = Ver[:]
    return Vertices,start,goal

#Search a node in the list of vertices by coordinate (x,y)
def Search(x,y,Vert): 
    for i in Vert:
        if x == i[0] and y == i[1]:
            return i
    return False

#Generator of Graph
def Graph(Vert,L):
    G = []                       #If we search in the maze looking in the right node
    A = []                       #and the down node we can find all the possibles neighbors in the graph
    N2 = []                       
    for i in Vert:
        x,y = i[0],i[1]
        N2 = Search(x,y+1,Vert)     #Search node in the down from the current position
        if N2 != False:
            A.append(Node(i,None))
            A.append(Node(N2,None))
            #A.append(1)
            G.append(A)
        A = []
        N2 = Search(x+1,y,Vert)     #Search a node in the right of the current position
        if N2 != False:
            A.append(Node(i,None))
            A.append(Node(N2,None))
            #A.append(1)
            G.append(A)
        A = []
    return G

def GraphStart(V,L):
    G = []                       #Si revisamos el mapa buscando las casillas de la derecha 
    A = []                       #y la de abajo podremos encontrar todos los posibles vecinos del grafo
    N2 = []                       
    for i in V:
        x,y = i[0],i[1]
        N2 = Search(x,y+1,V)     #Buscamos si hay un nodo abajo
        if N2 != False:
            A.append(NodeStar(i,None,0,0))
            A.append(NodeStar(N2,None,0,0))
            #A.append(1)
            G.append(A)
        A = []
        N2 = Search(x+1,y,V)     #Buscamos si hay un nodo a la derecha
        if N2 != False:
            A.append(NodeStar(i,None,0,0))
            A.append(NodeStar(N2,None,0,0))
            #A.append(1)
            G.append(A)
        A = []
    return G

#Function to read input data from Hacker Rank
def inputData():
    start = input()
    goal = input()
    mazeMap = input()
    mazeMap = mazeMap.split()
    for i in range(int(mazeMap[0])):
        L.append(input())
    return start, goal, L
        
#Verify if pacman already pass on this node searching in the list of nodes
def Seen(Node,Seen):
    if len(Seen) != 0:
        for i in Seen:
            if Node.position == i.position:
                return False
    return True

#Find the node with the min f(n) value and return his position
def ExtractNode(border):
    e = border[0]
    for n in border:
        if n < e:
            e = n
    i = border.index(e)
    return i

#Search the neighbors of a node in the graph
def findNeighbors(actualNode,G,border):
    neighbors = []
    for i in G:
        if i[0].position == actualNode.position:
            neighbors.append(Node(i[1].position,actualNode))
        elif i[1].position == actualNode.position:
            neighbors.append(Node(i[0].position,actualNode))
    #Verificar si algun vecino ya esta en la frontera para no meterlo
    for i in neighbors:
        if Seen(i,border):
            border.append(i)
            
def findStarNeighbors(Node,G,border):
    vecinos = []
    for i in G:
        if i[0].position == Node.position:
            vecinos.append(NodeStar(i[1].position,Node,Node.g+1,i[1].hn(goal)))
        elif i[1].position == Node.position:
            vecinos.append(NodeStar(i[0].position,Node,Node.g+1,i[0].hn(goal)))
    #Verificar si algun vecino ya esta en la frontera para no meterlo
    for i in vecinos:
        if Seen(i,border):
            border.append(i)

#Generate a list with the path from the start to the goal in the maze
def Path(seen,path):
    n = seen[:]
    e = n[len(n)-1]
    while e.father != None:
        e = e.father
        path.append(e)
    path.reverse()
    return path

In [3]:
def DFS(G,start,goal):
    border = []
    seen = []
    path = []
    border.append(start)
    while border:
        actualNode = border.pop(len(border)-1) #Extrack nodo from stack
        if actualNode.position == goal.position:
            seen.append(actualNode)
            path.append(actualNode)
            break
        else:
            if Seen(actualNode,seen):
                seen.append(actualNode)
                findNeighbors(actualNode,G,border)
    return seen,Path(seen,path)

def BFS(G,start,goal):
    border = []
    seen = []
    path = []
    border.append(start)
    while border:
        actualNode = border.pop(0) #Extraer nodo de la cola
        if actualNode.position == goal.position:
            seen.append(actualNode)
            path.append(actualNode)
            break
        else:
            #Checar si el nodo no se ha visitado ya para buscar sus vecinos
            if Seen(actualNode,seen):
                seen.append(actualNode)
                findNeighbors(actualNode,G,border)
    return seen,Path(seen,path)

def AStar(G,start,goal):
    border = []
    seen = []
    path = []
    border.append(start)
    while border:
        Node = border.pop(ExtractNode(border)) #Extraer el nodo con menor f(n)
        if Node.position == goal.position:
            seen.append(Node)
            path.append(Node)
            break
        else:
            #Checar si el nodo no se ha visitado ya para buscar sus vecinos
            if Seen(Node,seen):
                seen.append(Node)
                findStarNeighbors(Node,G,border)
    return Path(seen,path)

In [15]:
if __name__ == '__main__':
    V = []
    G = []
    start = []
    goal = []
    L = []
    algorithms = {"AStar": 1, "BFS": 2, "DFS": 3}
    algorithm = algorithms["AStar"]
    input_file = 'mazeH2.txt'
    L = open(input_file).readlines()
    #start, goal, L = inputData()
    V,start,goal = Vertices(L)
    if(algorithm == 1):
        G = GraphStart(V,L)
        path = AStar(G,NodeStar(start,None,0,0),NodeStar(goal,None,0,0))
        print(len(path)-1)
        for j in path:
            print(j)
    else:
        G = Graph(V,L)
        if(algorithm == 2):
            seen,path = BFS(G,Node(start,None),Node(goal,None))
        else:
            seen,path = DFS(G,Node(start,None),Node(goal,None))
        print(len(seen))
        for i in seen:
            print(i)
        print(len(path)-1)
        for j in path:
            print(j)

15
11 9
11 10
11 11
11 12
11 13
10 13
9 13
8 13
7 13
6 13
5 13
4 13
3 13
3 14
3 15
2 15
